In [1]:
!pip install Bio
!pip install hmmlearn

In [2]:
from google.colab import drive
import psutil
import time

drive.mount('/content/drive')
input_file = '/content/drive/My Drive/IIR_orientation/HW6/GRCh38_latest_genomic.fna'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from Bio import SeqIO
fasta_sequences = SeqIO.to_dict(SeqIO.parse(open(input_file),'fasta'))


In [4]:
S = str(fasta_sequences['NC_000006.12'].seq)[100000:200000]
S = S.lower()
S

'...'

In [5]:
def printResult(log_base2_prob, EXECUTION_START_TIME, EXECUTION_START_MEM, EXECUTION_END_TIME, EXECUTION_END_MEM):
  print('log base 2 probability:', log_base2_prob)
  print('running time:', EXECUTION_END_TIME-EXECUTION_START_TIME)
  print('memory usage:', EXECUTION_END_MEM-EXECUTION_START_MEM)

# Markov Model Order0

In [6]:
#order0
from collections import Counter
import pandas as pd
import numpy as np

EXECUTION_START_TIME = time.time()
EXECUTION_START_MEM = psutil.Process().memory_info().rss

def parse_log2(x):
  if x == 'a':
    return np.log2(Prob_a)
  elif x == 't':
    return np.log2(Prob_t)
  elif x == 'c':
    return np.log2(Prob_c)
  else:
    return np.log2(Prob_g)  

S_dict = Counter(S)

all = S_dict['a']+S_dict['t']+S_dict['c']+S_dict['g']
Prob_a = S_dict['a']/all
Prob_t = S_dict['t']/all
Prob_c = S_dict['c']/all
Prob_g = S_dict['g']/all

df_0 = pd.DataFrame({'S':list(S)})
df_0['Prob_log2'] = df_0['S'].apply(lambda x: parse_log2(x))
df_0



,S,Prob_log2
0,t,-1.895073
1,t,-1.895073
2,g,-2.265622
3,g,-2.265622
4,t,-1.895073
...,...,...
99995,g,-2.265622
99996,c,-2.265137
99997,c,-2.265137
99998,t,-1.895073


In [7]:
log_base2_prob = df_0['Prob_log2'].sum()
EXECUTION_END_TIME = time.time()
EXECUTION_END_MEM = psutil.Process().memory_info().rss
printResult(log_base2_prob, EXECUTION_START_TIME, EXECUTION_START_MEM, EXECUTION_END_TIME, EXECUTION_END_MEM)

log base 2 probability: -197688.94757656846
running time: 0.18749165534973145
memory usage: 798720


# Markov Model Order1

In [8]:
#order1

EXECUTION_START_TIME = time.time()
EXECUTION_START_MEM = psutil.Process().memory_info().rss

df_1 = pd.DataFrame({'S':list(S)})
df_1['S_prev'] = df_1['S'].shift(1)
df_1['freq'] = df_1['S_prev'].str.cat(df_1['S'])
df_1['S_prev_cnt']=df_1['S_prev'].map(dict(df_1['S_prev'].value_counts()))
df_1['freq_cnt']=df_1['freq'].map(dict(df_1['freq'].value_counts()))
df_1['Prob_log2']=np.log2(df_1['freq_cnt']/df_1['S_prev_cnt'])
df_1

,S,S_prev,freq,S_prev_cnt,freq_cnt,Prob_log2
0,t,NaN,NaN,NaN,NaN,NaN
1,t,t,tt,26886.0,7909.0,-1.765288
2,g,t,tg,26886.0,7190.0,-1.902791
3,g,g,gg,20795.0,5358.0,-1.956470
4,t,g,gt,20795.0,4744.0,-2.132061
...,...,...,...,...,...,...
99995,g,t,tg,26886.0,7190.0,-1.902791
99996,c,g,gc,20795.0,4412.0,-2.236732
99997,c,c,cc,20803.0,5308.0,-1.970551
99998,t,c,ct,20803.0,6656.0,-1.644064


In [9]:
log_base2_prob = df_1['Prob_log2'].sum()+df_0.loc[0].Prob_log2
EXECUTION_END_TIME = time.time()
EXECUTION_END_MEM = psutil.Process().memory_info().rss
printResult(log_base2_prob, EXECUTION_START_TIME, EXECUTION_START_MEM, EXECUTION_END_TIME, EXECUTION_END_MEM)

log base 2 probability: -193211.25299991065
running time: 0.13922715187072754
memory usage: 2097152


In [10]:
#order2

EXECUTION_START_TIME = time.time()
EXECUTION_START_MEM = psutil.Process().memory_info().rss

df_2 = pd.DataFrame({'S':list(S)})
df_2['S_prevL'] = df_2['S'].shift(2)
df_2['S_prevR'] = df_2['S'].shift(1)
df_2['S_prev'] = df_2['S_prevL'].str.cat(df_2['S_prevR'])
df_2['freq'] = df_2['S_prev'].str.cat(df_2['S'])

df_2['S_prev_cnt']=df_2['S_prev'].map(dict(df_2['S_prev'].value_counts()))
df_2['freq_cnt']=df_2['freq'].map(dict(df_2['freq'].value_counts()))
df_2['Prob_log2']=np.log2(df_2['freq_cnt']/df_2['S_prev_cnt'])
df_2

,S,S_prevL,S_prevR,S_prev,freq,S_prev_cnt,freq_cnt,Prob_log2
0,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,t,NaN,t,NaN,NaN,NaN,NaN,NaN
2,g,t,t,tt,ttg,7909.0,1591.0,-2.313561
3,g,t,g,tg,tgg,7189.0,1898.0,-1.921311
4,t,g,g,gg,ggt,5358.0,1107.0,-2.275039
...,...,...,...,...,...,...,...,...
99995,g,t,t,tt,ttg,7909.0,1591.0,-2.313561
99996,c,t,g,tg,tgc,7189.0,1454.0,-2.305764
99997,c,g,c,gc,gcc,4412.0,1248.0,-1.821815
99998,t,c,c,cc,cct,5308.0,1750.0,-1.600813


In [11]:
log_base2_prob = df_2['Prob_log2'].sum()+df_0.loc[0].Prob_log2+df_1.loc[1].Prob_log2
EXECUTION_END_TIME = time.time()
EXECUTION_END_MEM = psutil.Process().memory_info().rss
printResult(log_base2_prob, EXECUTION_START_TIME, EXECUTION_START_MEM, EXECUTION_END_TIME, EXECUTION_END_MEM)

log base 2 probability: -192098.42695359018
running time: 0.18552637100219727
memory usage: 12959744


# Hidden Markov Model

In [12]:
from hmmlearn import hmm

np.random.seed(1)
hidden_states = ['s1', 's2', 's3']
S_input = [int(i) for i in list(S.replace('a','0').replace('t','1').replace('c','2').replace('g','3'))]
model = hmm.MultinomialHMM(n_components=len(hidden_states), n_iter=100, tol=0.01)
model.fit(np.array(S_input).reshape(-1,1))

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=3,
               n_iter=100, params='ste',
               random_state=RandomState(MT19937) at 0x7FF634DD2050,
               startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
               verbose=False)

In [13]:
print('Initial Probability:', model.startprob_)
print('Transition Matrix:', model.transmat_)
print('Emission Matrix:', model.emissionprob_)
print('log base exponential probability:', model.score(np.array(S_input).reshape(-1,1)))

Initial Probability: [1.00000000e+00 4.36444125e-42 5.66304472e-24]
Transition Matrix: [[0.11108823 0.57355919 0.31535258]
 [0.28404443 0.34527653 0.37067903]
 [0.5207415  0.01918942 0.46006908]]
Emission Matrix: [[4.77890427e-01 4.73176681e-01 1.52508511e-05 4.89176414e-02]
 [1.08446697e-01 2.20852965e-01 2.67154155e-02 6.43984922e-01]
 [3.36363775e-01 1.36143645e-01 5.16576901e-01 1.09156790e-02]]
log base exponential probability: -134342.80085634952


In [14]:
EXECUTION_START_TIME = time.time()
EXECUTION_START_MEM = psutil.Process().memory_info().rss

S_input = [int(i) for i in list(S.replace('a','0').replace('t','1').replace('c','2').replace('g','3'))]
log_base2_prob = model.score(np.array(S_input).reshape(-1,1))/np.log(2)

EXECUTION_END_TIME = time.time()
EXECUTION_END_MEM = psutil.Process().memory_info().rss

printResult(log_base2_prob, EXECUTION_START_TIME, EXECUTION_START_MEM, EXECUTION_END_TIME, EXECUTION_END_MEM)

log base 2 probability: -193815.69257458905
running time: 0.1975104808807373
memory usage: 98304


# Another 100kb section of the human genome

In [15]:
EXECUTION_START_TIME = time.time()
EXECUTION_START_MEM = psutil.Process().memory_info().rss

new_S = str(fasta_sequences['NC_000006.12'].seq)[200000:302000]
new_S = S.lower()
new_S_input = [int(i) for i in list(new_S.replace('a','0').replace('t','1').replace('c','2').replace('g','3'))]

In [16]:
logprob, state_seq = model.decode(np.array(new_S_input).reshape(-1,1), algorithm='viterbi')

In [17]:
log_base2_prob = logprob/np.log(2)

In [18]:
EXECUTION_END_TIME = time.time()
EXECUTION_END_MEM = psutil.Process().memory_info().rss
printResult(log_base2_prob, EXECUTION_START_TIME, EXECUTION_START_MEM, EXECUTION_END_TIME, EXECUTION_END_MEM)

log base 2 probability: -237610.46483163847
running time: 0.19103145599365234
memory usage: 0


In [19]:
state_seq

array([0, 1, 1, ..., 2, 0, 1], dtype=int32)